# March Madness Machine Learning Project

Autor: Dominik Babić

## Reprodukcija rezultata eksperimenata


Navedeni rad kao pristup rješavanja problema predviđanja vjerojatnosti pobjede određene ekipe uzima individualno uvažavanje varijabli koje statistički najviše koreliraju s postotkom pobjede, izračunavanje kvadratne regresijske funkcije nad njima te zatim očekivanje vjerojatni pobjede nad skupom izračunatih regresijskih vrijednosti.

Značajke koje su u radu određene kao najznačajnije su:
- Razlika u **RPI** rangiranju (*Rating Percentage Index*)
- Razlika u **BPI** (*Basketball Power Index*)
- Razlika u ukupnom broju poena (Razlika u ukupnom broju poena)
- Razlika u ukupnom broju ukradenih lopti (Razlika u ukupnom broju ukradenih lopti)
- Razlika u ukupnom broju blokada (Razlika u ukupnom broju blokada)
- Razlika u postotku šuta iz igre (Razlika u postotku šuta iz igre)

To su značajke koje su dostupne prije samih odigranih utakmica, tj. predstavljaju statistike timova kroz njihovu regularnu sezonu, što ih čini idealnim za korištenje pri predviđanju pobjednika. 

Naime, podaci o samoj utakmici nam neće pomoći jer su oni dostupni tek nakon što je utakmica odigrana, a cilj nam je predvidjeti pobjednika prije samog početka utakmice.

Ovo znanje iskoristit ćemo pri replikaciji eksperimenata iz rada.

In [123]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import seaborn as sns

Prvo ćemo izvući navedene značajke iz našeg skupa podataka te prikazati njihovu korelaciju s postotkom pobjede.
U tu svrhu iskoristit ćemo `Tournament Team Data.csv` i `2023 Tournament Data.csv` datoteke za treniranje te `Game Data.csv` `Tournament Team Data (Including 2023).csv` za testiranje modela s obzirom da želimo izvesti predviđanja za timove koji su se kvalificirali u sezoni 2022./2023.

BPI vrijednosti za ekipe kroz godine dohvaćene su s ESPN-ovog API-ja te su spremljene u `bpi_data.csv` datoteku.

In [251]:
tournament_data_2023 = pd.read_csv('march-madness-data/2023 Tournament Data.csv')
tournament_team_data_2023 = pd.read_csv('march-madness-data/Tournament Team Data (Including 2023).csv')
tournament_team_data_2023 = tournament_team_data_2023[tournament_team_data_2023['YEAR'] == 2023]
tournament_team_data = pd.read_csv('march-madness-data/Tournament Team Data.csv')
game_data = pd.read_csv('march-madness-data/Game Data.csv')
bpi_data = pd.read_csv('march-madness-data/bpi_data.csv')

# Nekonzistentnost podataka skupljenih za BPI index s nazivima timova u ostalim skupovima
# bpi_data['TEAM'] = bpi_data['TEAM'].replace(r'\b(\w+\s*St)\b', r'\1.', regex=True)
# bpi_data['TEAM'] = bpi_data['TEAM'].str.replace('State', 'St.')
# bpi_data['TEAM'] = bpi_data['TEAM'].str.replace(r'\bMiami\b', 'Miami FL', regex=True)
# bpi_data['TEAM'] = bpi_data['TEAM'].replace('Fullerton', 'Cal St. Fullerton')
# bpi_data['TEAM'] = bpi_data['TEAM'].replace('Jacksonville', 'Jacksonville St.')
# bpi_data['TEAM'] = bpi_data['TEAM'].replace('St Bonaventure', 'St. Bonaventure')
# bpi_data['TEAM'] = bpi_data['TEAM'].replace('Texas A&M-CC', 'Texas A&M CC')
# bpi_data['TEAM'] = bpi_data['TEAM'].replace('Santa Barbara', 'UC Santa Barbara')
# bpi_data['TEAM'] = bpi_data['TEAM'].replace('Abilene Chrstn', 'Abilene Christian')
# bpi_data['TEAM'] = bpi_data['TEAM'].replace('E Washington', 'Eastern Washington')
# bpi_data['TEAM'] = bpi_data['TEAM'].replace('Mount St. Marys', "Mount St. Mary's")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace('Virginia ', "Virginia")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("St John's", "St. John's")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("N Kentucky", "Northern Kentucky")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Fair Dickinson", "Fairleigh Dickinson")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Gardner-Webb", "Gardner Webb")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Gardner-Webb", "Gardner Webb")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("N Dakota St.", "North Dakota St.")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("NC Central", "North Carolina Central")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("NC St.", "North Carolina St.")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Prairie View", "Prairie View A&M")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Charleston", "College of Charleston")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("SF Austin", "Stephen F. Austin")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Long Island", "LIU Brooklyn")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("MTSU", "Middle Tennessee")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("ETSU", "East Tennessee St.")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("FGCU", "Florida Gulf Coast")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Pitt", "Pittsburgh")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Little Rock", "Arkansas Little Rock")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Hawai'i", "Hawaii")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Bakersfield", "Cal St. Bakersfield")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("UAlbany", "Albany")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Coastal Car", "Coastal Carolina")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("UMass", "Massachusetts")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("G Washington", "George Washington")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("W Michigan", "Western Michigan")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Louisiana", "Louisiana Lafayette")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("E Kentucky", "Eastern Kentucky")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("N'Western St.", "Northwestern St.")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("NC A&T", "North Carolina A&T")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Western KY", "Western Kentucky")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Detroit Mercy", "Detroit")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Miss Valley St.", "Mississippi Valley St.")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("N Colorado", "Northern Colorado")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Boston U", "Boston University")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("Sam Houston", "Sam Houston St.")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("AR-Pine Bluff", "Arkansas Pine Bluff")
# bpi_data['TEAM'] = bpi_data['TEAM'].replace("CSU Northridge", "Cal St. Northridge")
# bpi_data.to_csv('march-madness-data/bpi_data.csv', index=False)

tournament_team_data['TEAM'] = tournament_team_data['TEAM'].replace("Virginia ", "Virginia")
tournament_team_data['TEAM'] = tournament_team_data['TEAM'].replace("Mississippi", "Mississippi St.")

tournament_data_2023 = tournament_data_2023.loc[:, ['YEAR', 'SEED', 'TEAM', 'WIN %', 'BLOCK %', 'BARTHAG', 'POINTS PER POSSESSION DEFENSE', 'POINTS PER POSSESSION OFFENSE']]
tournament_data_2023 = tournament_data_2023.merge(bpi_data, on=['YEAR', 'TEAM'], how='left')

tournament_team_data = tournament_team_data.loc[:, ['YEAR', 'SEED', 'TEAM', 'WIN %', 'BLOCK %', 'BARTHAG', 'POINTS PER POSSESSION DEFENSE', 'POINTS PER POSSESSION OFFENSE']]
tournament_team_data = tournament_team_data.merge(bpi_data, on=['YEAR', 'TEAM'], how='left')

tournament_team_data.info()
tournament_team_data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 943 entries, 0 to 942
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   YEAR                           943 non-null    int64  
 1   SEED                           943 non-null    int64  
 2   TEAM                           943 non-null    object 
 3   WIN %                          943 non-null    float64
 4   BLOCK %                        943 non-null    float64
 5   BARTHAG                        943 non-null    float64
 6   POINTS PER POSSESSION DEFENSE  943 non-null    float64
 7   POINTS PER POSSESSION OFFENSE  943 non-null    float64
 8   BPI                            943 non-null    float64
dtypes: float64(6), int64(2), object(1)
memory usage: 66.4+ KB


,YEAR,SEED,WIN %,BLOCK %,BARTHAG,POINTS PER POSSESSION DEFENSE,POINTS PER POSSESSION OFFENSE,BPI
count,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000,943.000000
mean,2014.696713,8.766702,72.545270,10.565536,0.797949,0.972245,1.087409,9.342842
std,4.238543,4.673941,9.958894,2.909901,0.168831,0.044878,0.049885,6.176630
min,2008.000000,1.000000,36.670000,3.700000,0.129000,0.834000,0.907000,-16.000000
25%,2011.000000,5.000000,65.630000,8.500000,0.743500,0.942000,1.054000,5.800000
50%,2015.000000,9.000000,72.730000,10.300000,0.861000,0.974000,1.087000,10.000000
75%,2018.000000,13.000000,78.790000,12.500000,0.916000,1.003000,1.122500,13.550000
max,2022.000000,16.000000,100.000000,21.400000,0.985000,1.119000,1.241000,23.900000


Sada ćemo pokušati izvesti navedene značajke iz našeg skupa podataka te prikazati njihovu korelaciju s postotkom pobjede. Možemo primjetiti da neke od navedenih značajki nisu dostupne u skupu podataka, a nije ih moguće izračunati iz dostupnih podataka, one će biti zamijenjene sličnim značajkama koje su dostupne u skupu podataka (nisu dostupni BPI, RPI, ukupni blokovi, ukupni poeni i broj ukradenih lopti).

Prvi korak je određivanje pobjednika i gubitnika utakmica čije podatke imamo u skupu podataka `Game Data.csv`. U tom skupu podataka svaka dva retka prikazuju podatke o jednoj utakmici, tako da ćemo reducirati broj podataka na pola.

In [252]:
new_columns = ['YEAR', 'WINNING TEAM', 'LOSING TEAM', 'W TEAM LOC', 'SCORE DIFF']

new_game_data_rows = []

for i in range(0, len(game_data), 2):
    game1_row = game_data.iloc[i]
    game2_row = game_data.iloc[i + 1]

    winning_team_stats = game1_row if game1_row['TEAM 1 OUTCOME'] == 'W' else game2_row
    losing_team_stats = game2_row if game1_row['TEAM 1 OUTCOME'] == 'W' else game1_row

    new_row = [game1_row['YEAR'], winning_team_stats['TEAM 1'], losing_team_stats['TEAM 1'], winning_team_stats['TEAM 1 LOCATION'], winning_team_stats['TEAM 1 SCORE'] - losing_team_stats['TEAM 2 SCORE']]
    new_game_data_rows.append(new_row)

game_data = pd.DataFrame(new_game_data_rows, columns=new_columns)

In [253]:
game_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80113 entries, 0 to 80112
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   YEAR          80113 non-null  int64 
 1   WINNING TEAM  80113 non-null  object
 2   LOSING TEAM   80113 non-null  object
 3   W TEAM LOC    80113 non-null  object
 4   SCORE DIFF    80113 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 3.1+ MB


Sada ćemo u ovu tablicu nadodati podatke pobjednika i gubitnika za istu godinu iz tablice `Tournament Team Data.csv` te izračunati razliku u svim značajkama koje su nam dostupne. Prvo bismo htjeli pogledati koliko utakmica sadrži timove čije podatke uopće ne znamo.

In [255]:
not_in_tournament = game_data[~game_data['WINNING TEAM'].isin(tournament_team_data['TEAM']) | ~game_data['LOSING TEAM'].isin(tournament_team_data['TEAM'])]
count = len(not_in_tournament)
print(f"The number of TEAM names in game_data that are not in tournament_team_data is: {count}")

both_in_tournament = game_data[game_data['WINNING TEAM'].isin(tournament_team_data['TEAM']) & game_data['LOSING TEAM'].isin(tournament_team_data['TEAM'])]
count = len(both_in_tournament)
print(f"The number of TEAM names in game_data that are in tournament_team_data is: {count}")

The number of TEAM names in game_data that are not in tournament_team_data is: 40192
The number of TEAM names in game_data that are in tournament_team_data is: 39921


Broj utakmica u kojima su sudjelovali oba tima čije statistike su poznate je dovoljno velik da nam nije potrebno popunjavanje nepostojećih vrijednosti, tako da ćemo ih jednostavno izbaciti iz skupa podataka.

In [308]:
merged_data_winning = pd.merge(game_data, tournament_team_data, left_on=['YEAR', 'WINNING TEAM'], right_on=['YEAR', 'TEAM'], how='outer')
merged_data_winning = merged_data_winning.drop(["TEAM"], axis=1)
merged_data_winning = merged_data_winning.rename(columns=lambda x: 'WINNER TEAM ' + x if x not in ['YEAR', 'LOSING TEAM', 'WINNING TEAM', 'W TEAM LOC', 'SCORE DIFF'] else x)

merged_data_losing = pd.merge(game_data, tournament_team_data, left_on=['YEAR', 'LOSING TEAM'], right_on=['YEAR', 'TEAM'], how='outer')
merged_data_losing = merged_data_losing.drop(["TEAM"], axis=1)
merged_data_losing = merged_data_losing.rename(columns=lambda x: 'LOSER TEAM ' + x if x not in ['YEAR', 'WINNING TEAM', 'LOSING TEAM', 'W TEAM LOC', 'SCORE DIFF'] else x)

merged_data = pd.merge(merged_data_winning, merged_data_losing, on=['YEAR', 'WINNING TEAM', 'LOSING TEAM', 'W TEAM LOC', 'SCORE DIFF'], how='outer')

# Filter out non-numeric columns
numeric_cols = merged_data.select_dtypes(include=[np.number]).columns

for col in numeric_cols.values:
    if merged_data[col].isnull().values.any():
        merged_data[col].fillna(merged_data[col].interpolate(), inplace=True)

merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80230 entries, 0 to 80229
Data columns (total 19 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   YEAR                                       80230 non-null  int64  
 1   WINNING TEAM                               80227 non-null  object 
 2   LOSING TEAM                                80227 non-null  object 
 3   W TEAM LOC                                 80227 non-null  object 
 4   SCORE DIFF                                 80230 non-null  float64
 5   WINNER TEAM SEED                           80188 non-null  float64
 6   WINNER TEAM WIN %                          80188 non-null  float64
 7   WINNER TEAM BLOCK %                        80188 non-null  float64
 8   WINNER TEAM BARTHAG                        80188 non-null  float64
 9   WINNER TEAM POINTS PER POSSESSION DEFENSE  80188 non-null  float64
 10  WINNER TEAM POINTS PER

Kao što se može primjetiti, zbog desnog spajanja podataka, sada imamo puno manje zabilježenih utakmica, a razlog tome je taj što neke ekipe nisu sudjelovale u turniru, a neke ekipe nisu imale podatke o nekim značajkama. U ovom slučaju, te ekipe neće biti uzete u obzir pri treniranju modela.

In [310]:
# Calculate OWP
owp_data = merged_data.groupby(['YEAR', 'WINNING TEAM']).agg({'LOSER TEAM WIN %': 'mean'}).reset_index()
owp_data = owp_data.rename(columns={'LOSER TEAM WIN %': 'OWP', 'WINNING TEAM': 'TEAM'})

# Calculate OOWP
oowp_data = owp_data.groupby(['YEAR', 'TEAM']).agg({'OWP': 'mean'}).reset_index()
oowp_data = oowp_data.rename(columns={'OWP': 'OOWP'})

# Merge OWP and OOWP data with merge_data
merged_data = pd.merge(owp_data, merged_data, left_on=['YEAR', 'TEAM'], right_on=['YEAR', 'WINNING TEAM'], how='left')
merged_data = pd.merge(oowp_data, merged_data, left_on=['YEAR', 'TEAM'], right_on=['YEAR', 'WINNING TEAM'], how='left')

# Calculate RPI index
merged_data['RPI Index WINNING TEAM'] = (0.25 * merged_data['WINNER TEAM WIN %']) + (0.50 * merged_data['OWP']) + (0.25 * merged_data['OOWP'])

# Calculate OWP for LOSING TEAM
owp_data_losing = merged_data.groupby(['YEAR', 'LOSING TEAM']).agg({'WINNER TEAM WIN %': 'mean'}).reset_index()
owp_data_losing = owp_data_losing.rename(columns={'WINNER TEAM WIN %': 'OWP', 'LOSING TEAM': 'TEAM'})

# Calculate OOWP for LOSING TEAM
oowp_data_losing = owp_data_losing.groupby(['YEAR', 'TEAM']).agg({'OWP': 'mean'}).reset_index()
oowp_data_losing = oowp_data_losing.rename(columns={'OWP': 'OOWP'})

# Merge OWP and OOWP data with merged_data for LOSING TEAM
merged_data = pd.merge(owp_data_losing, merged_data, left_on=['YEAR', 'TEAM'], right_on=['YEAR', 'LOSING TEAM'], how='left', suffixes=('_owp_losing', '_merged'))
merged_data = pd.merge(oowp_data_losing, merged_data, left_on=['YEAR', 'TEAM'], right_on=['YEAR', 'LOSING TEAM'], how='left', suffixes=('_owp_losing', '_merged'))

merged_data.info()
# Calculate RPI index for LOSING TEAM
# merged_data['RPI Index LOSING TEAM'] = (0.25 * merged_data['LOSER TEAM WIN %']) + (0.50 * merged_data['OWP']) + (0.25 * merged_data['OOWP'])

# merged_data.info()
# merged_data.describe()

MergeError: Passing 'suffixes' which cause duplicate columns {'TEAM_y'} is not allowed.